

\begin{equation}
\frac{1}{2} (\partial_t v_x \sigma_x + \partial_t v_y \sigma_y
+\partial_t v_z \sigma_z) = - \gamma (v_x \sigma_x + v_y \sigma_y)
\end{equation}

which is solved by

\begin{align}
v_x(t) &= v_x(0)e^{-2\gamma t} \\
v_y(t) &= v_y(0)e^{-2\gamma t} \\
v_z(t) &= v_z(0)e^{-2\gamma t} \\
\end{align}